In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Generate a random matrix A
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A

In [3]:
sample_size = 100000

diag_estimate = np.zeros(n)
tk = np.zeros(n)
qk = np.zeros(n)

for j in range(sample_size):
    
    # Draw random vector
    vk = np.random.choice([-1, 1], size=n)

    # Update tk
    tk = tk + ((A @ vk) * vk)

    # Update qk
    qk = qk + (vk*vk)

    # Update diag_estimate
    diag_estimate = tk / qk

In [4]:
(np.linalg.norm( np.diag(A) - diag_estimate , ord=2)**2)/n

9.864831662658363

In [ ]:
def naive_diag(A, sample_size=1000):
    """Naive unbiased estimator for the diagonal of a matrix, see [5].
    """

    # Get shape
    n = A.shape[0]

    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = np.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

# Explicit probe?

In [10]:
def explicit_diag_probe(A):
    """Computes the diagonal of A using an explicit probe. Requires exactly n matvecs and rmatvecs with A.
    """

    # Setup
    n = A.shape[0]
    diagonal = np.zeros(n)

    for j in range(n):

        # jth column of the identity
        w = np.zeros(n)
        w[j] = 1.0

        # Compute w^T A w
        diagonal[j] = w.T @ A @ w

    return diagonal

In [12]:
explicit_diag_probe(A) - np.diag(A)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Hadamard matrices method?